### Analyze a single event without LArbysImage

In [ ]:
import sys,os,tempfile

from larcv import larcv
import ROOT
from ROOT import larocv

import matplotlib
matplotlib.rcParams['font.size']=20
matplotlib.rcParams['font.family']='serif'
from matplotlib import pyplot as plt
import numpy as np

from ROOT import geo2d
pygeo = geo2d.PyDraw()

%matplotlib inline

### ProcessDriver configuration

In [ ]:
INFILE = "/tmp/one_event_test.root"
CFG  = os.path.join(os.environ['LARCV_BASEDIR'],"app/LArOpenCVHandle/cfg/develop_external.cfg")
proc = larcv.ProcessDriver('ProcessDriver')

proc.configure(CFG)
proc.override_input_file(ROOT.std.vector("string")(1,ROOT.std.string(INFILE)))
proc.initialize()  
iom      = proc.io()
ext_id   = proc.process_id("LArbysImageExtract")                                                                                                                          
ext_proc = proc.process_ptr(ext_id) 

### Entry selection

In [ ]:
event=0
proc.process_entry(event)

### Using LArbysImageExtract (ext_proc)
LArbysImageExtract is a module which gives the ability to access the LArOpenCV cv::Mat image in the same way as fed to the algorithm.

Call **ext_proc.FillcvMat()** to take the ADC, Track, Shower, ThruMu, StopMu larcv::Image2D data and convert to cv::Mat. Optionally you can call **ext_proc.FillcvMat(larcv::ROI)** to crop the images to the given ROI

In [ ]:
ext_proc.FillcvMat()
fig,ax=plt.subplots(figsize=(20,20))
plane=2
adc_mat=ext_proc.ADCImage(plane)
adc_mat=larocv.Threshold(adc_mat,10,255)
tmi = pygeo.image(adc_mat)
plt.imshow(tmi.T,cmap='Greys',interpolation='none')
plt.show()

Here i'll get the first ROI by getting it from IOManager (**iom**), then call **FillcvMat(...)** again, this time specifying an ROI.

The result is displayed

In [ ]:
roi = iom.get_data(larcv.kProductROI,"croimerge").ROIArray()[0]
ext_proc.FillcvMat(roi)
print "Cropping to",roi

In [ ]:
for plane in xrange(3):
    fig,ax=plt.subplots(figsize=(20,20))
    mat=ext_proc.ADCImage(plane)
    mat = larocv.Threshold(mat,10,255)
    #for contour in larocv.FindContours(mat):
    #    ctor = np.array([[pt.x,pt.y] for pt in contour])
    #    plt.plot(ctor[:,0],ctor[:,1],lw=5)
    tmi = pygeo.image(mat)
    ax.imshow(tmi,cmap='Greys',interpolation='none')
    ax.grid()
    ax.set_title("Plane: %d"%plane,fontsize=30)
    plt.show()

## Side by side
For fun I can put the track, shower, thrumu, stopmu images side by side

In [ ]:
plane=1
fig,ax = plt.subplots(figsize=(20,20))

trk_mat = ext_proc.TrackImage(plane)
trk_mat = larocv.Threshold(trk_mat,10,150)
trk_img = pygeo.image(trk_mat)

shr_mat = ext_proc.ShowerImage(plane)
shr_mat = larocv.Threshold(shr_mat,10,80)
shr_img = pygeo.image(shr_mat)

tmu_mat = ext_proc.ThruMuImage(plane)
tmu_img = pygeo.image(tmu_mat)

smu_mat = ext_proc.StopMuImage(plane)
smu_img = pygeo.image(smu_mat)

tmu_img = np.where(tmu_img>0.0,0.0,1.0)
smu_img = np.where(smu_img>0.0,0.0,1.0)

comb_img = trk_img + shr_img
comb_img *= tmu_img
comb_img *= smu_img

ax.imshow(comb_img,cmap='jet',interpolation='none',vmin=1,vmax=255)

ax.grid()
ax.set_title("Plane: %d"%plane,fontsize=30)
plt.show()